In [53]:
import json
from pathlib import Path
from shutil import rmtree

In [54]:
from mmengine.config import Config
from mmpretrain.utils import load_json_log

In [55]:
WORKDIRS = '/home/zh21/mmpretrain/work_dirs'

In [56]:
## Clean up
for exp in Path(WORKDIRS).glob('*'):
    if not exp.is_dir():
        continue
    for trial in exp.glob('*'):
        if not trial.is_dir():
            continue
        config_path = trial / 'vis_data' / 'config.py'
        json_file = trial / 'vis_data' / f'{trial.stem}.json'
        json_file2 = trial / 'vis_data' / f'scalars.json'
        valid = config_path.exists() and ( json_file.exists() or json_file2.exists())
        if not valid:
            rmtree(trial)
            print(f'Delete {trial}')
        
        wandb_path = trial / 'vis_data' /'wandb'
        if wandb_path.exists():
            rmtree(wandb_path)
        
        for d in trial.glob('*'):
            if d.is_dir() and len(list(d.glob('*'))) == 0:
                d.rmdir()
                print(f'Delete {d}')

Delete /home/zh21/mmpretrain/work_dirs/clip_vit_b32/20230416_130735


In [57]:
ES = []
for exp in Path(WORKDIRS).glob('*'):
    if not exp.is_dir():
        continue
    for trial in exp.glob('*'):
        if not trial.is_dir():
            continue
        config_path = trial / 'vis_data' / 'config.py'
        json_file = trial / 'vis_data' / f'{trial.stem}.json'
        json_file2 = trial / 'vis_data' / f'scalars.json'
        try:
            cfg = Config.fromfile(str(config_path))
        except:
            config_path.unlink()
            print(f'Delete {config_path}')

        try:
            results = load_json_log(json_file)
        except:
            print(f'Delete {json_file}')
            continue
        ES.append({
            'exp': exp.stem,
            'trial': trial.stem,
            'cfg': cfg,
            'results': results
        })

Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f96b58d3280>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/tempfile.py", line 440, in __del__
    self.close()
  File "/opt/conda/lib/python3.8/tempfile.py", line 436, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpegktsy2z/tmpd4ai3mhs.py'


Delete /home/zh21/mmpretrain/work_dirs/vitb32_eval/20230416_131043/vis_data/config.py


In [58]:
x = ES[0]

In [59]:
def shots(x):
    data_root = x['cfg'].train_dataloader.dataset.data_root
    if '_' not in data_root:
        return 99
    try:
        s = data_root.split('_')[1][5:]
    except:
        print(data_root)
    return int(s)

In [60]:
acc_top1 = lambda x: x['results']['val'][-1]['accuracy/top1']

In [61]:
full_model_ft = lambda x: 'peft' not in x['cfg'].model

In [62]:
box = []
for E in ES:
    fft = full_model_ft(E)
    s = shots(E)
    acc = acc_top1(E)
    box.append([fft, s, acc])

In [63]:
import pandas as pd

In [65]:
pd.DataFrame.from_records(box, columns=['full_model_ft', 'shots', 'acc_top1']).groupby(['full_model_ft', 'shots']).max()

acc_top1
full_model_ft shots           
False         1      48.345154
              4      53.309692
              16     69.444443
True          1      51.063831
              4      59.219860
              16     66.607567
              99     72.281326